In [1]:
# Package installations to work on WIRE

! pip install mamba
! mamba install gensim openai -y
! mamba install -c anaconda nltk -y
! mamba install -c conda-forge spacy -y
! mamba install -c conda-forge pyldavis -y
! mamba install -c conda-forge pypdf2 -y
! pip install transformers sentence-transformers
! pip install scikit-network
! pip install git+https://github.com/rwalk/gsdmm.git
! pip install gensim

! python -m spacy download en_core_web_sm


[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip
/bin/bash: line 1: mamba: command not found
/bin/bash: line 1: mamba: command not found
/bin/bash: line 1: mamba: command not found
/bin/bash: line 1: mamba: command not found
/bin/bash: line 1: mamba: command not found

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip
  Cloning https://github.com/rwalk/gsdmm.git to /tmp/pip-req-build-ezifjbpr
  Running command git clone --filter=blob:none --quiet https://github.com/rwalk/gsdmm.git /tmp/pip-req-build-ezifjbpr
  Resolved https://github.com/rwalk/gsdmm.git to commit 4ad1b6b6976743681ee4976b4573463d359214ee
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To updat

In [2]:
# imports

import csv
import json
import os
import re
from pprint import pprint
import time
import re
import unicodedata
import os
import sys
from contextlib import contextmanager
from copy import deepcopy
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import SpectralEmbedding
import pickle
import sknetwork as skn
from random import *

import matplotlib.pyplot as plt
import seaborn as sns


import numpy as np
import pandas as pd

import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess

import nltk
nltk.download(["names", "stopwords", "state_union", "twitter_samples", "movie_reviews", "averaged_perceptron_tagger", "vader_lexicon", "punkt", "wordnet"])
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words("english")
from nltk.sentiment.vader import SentimentIntensityAnalyzer
ps = nltk.porter.PorterStemmer()

from gsdmm import MovieGroupProcess

from sentence_transformers import SentenceTransformer

import spacy
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

from itertools import combinations


from collections import Counter
# set seed for reproducibility
# np.random.seed(493)

[nltk_data] Downloading package names to /home/jovyan/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package state_union to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package state_union is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Packag

## Organizing Transcript-Topics (Same as Before)
Final data saves with transcript, topic keywords, sentiment, and program

In [3]:
def transcript_cleaning(month):
    #import spacy stuff
    sid = SentimentIntensityAnalyzer()
    nlp = spacy.load("en_core_web_sm")
    NER = spacy.load("en_core_web_sm")

    tr_tp_st = [[], [], [], []]

    gl = pd.read_csv("foxGuestList.csv", encoding="windows-1252")
    fox = open("fox_text.txt", "r", encoding="windows-1252")
    fox = fox.readlines()
    for l in range(len(gl)):
        #######################################
        ##Turning transcript into df of lines##
        #######################################
        lines = pd.DataFrame(
            columns=["Date", "Start_Hour", "Program", "Title", "Speaker", "Line"]
        )
        # print(lines)
        transcript = fox[l]
        meta = gl.iloc[l, :]
        dt = meta["Date"]
        if month in str(dt):
            if randint(1, 100) <= 5:
                print(l)
            chunks = transcript.split("|")
            line = ""
            spkr = ""
            for c in chunks:
                if (
                    "Page" not in c[0:10]
                    and "...." not in c
                    and c[0:15] not in meta["Title"]
                ):
                    # print(c)
                    if ":" in c:
                        c = c.split(":")
                        if str.isupper(c[0]):
                            lines.loc[len(lines)] = [
                                meta["Date"],
                                meta["Start_Hour"],
                                meta["Program"],
                                meta["Title"],
                                spkr,
                                line,
                            ]
                            # print(spkr + ':' + line)
                            spkr = c[0]
                            line = c[1]
                    else:
                        line += c
            lines.loc[len(lines)] = [
                meta["Date"],
                meta["Start_Hour"],
                meta["Program"],
                meta["Title"],
                spkr,
                line,
            ]

            #########################
            ##Keyword and Sentiment##
            #########################

            lines['Line'] = lines['Line'].apply(lambda x: x.lower())
            lines["VADER"] = lines["Line"].apply(
                lambda x: list(sid.polarity_scores(x).values())[3]
            )

            word_sentiment_list = []
            w_list = []
            raw_text= ' '.join(lines['Line'])
            tokens = nltk.word_tokenize(raw_text)
            words = Counter(tokens).most_common()
            words1 = []
            i = 0
            while len(words1) < 15 and i < len(words):
                w = words[i][0]
                #print(w)
                tag =  nltk.pos_tag([w])
                t = tag[0][1] 
                if t in ['NN','NNS','NNP','NNPS','FW','SYM'] and w not in ['video','clip','i','end','begin']: #or t == "FW":
                    words1.append(w)
                i += 1

            for w in words1:
                try:
                    w1 = w
                    wdf = lines[lines['Line'].str.contains(w1)]
                    w_list.append(w1)
                    sent_list = list(wdf["VADER"])
                    word_sentiment_list.append(sent_list)
                except:
                    pass

                
            ######################
            ##Storing Each Topic##
            ######################
            for w in range(len(w_list)):
                if len(word_sentiment_list[w]) > 2:
                    tr_tp_st[0].append(l)
                    tr_tp_st[1].append(w_list[w])
                    tr_tp_st[2].append(np.mean(word_sentiment_list[w]))
                    tr_tp_st[3].append(lines["Program"][0])

    gl = pd.read_csv("msnbcGuestList.csv", encoding="windows-1252")
    fox = open("msnbc_text.txt", "r", encoding="windows-1252")
    fox = fox.readlines()
    for l in range(len(gl)):
        #######################################
        ##Turning transcript into df of lines##
        #######################################
        lines = pd.DataFrame(
            columns=["Date", "Start_Hour", "Program", "Title", "Speaker", "Line"]
        )
        # print(lines)
        transcript = fox[l]
        meta = gl.iloc[l, :]
        dt = meta["Date"]
        if month in str(dt):
            if randint(1, 100) <= 5:
                print(l)
            chunks = transcript.split("|")
            line = ""
            spkr = ""
            for c in chunks:
                if (
                    "Page" not in c[0:10]
                    and "...." not in c
                    and c[0:15] not in meta["Title"]
                ):
                    # print(c)
                    if ":" in c:
                        c = c.split(":")
                        if str.isupper(c[0]):
                            lines.loc[len(lines)] = [
                                meta["Date"],
                                meta["Start_Hour"],
                                meta["Program"],
                                meta["Title"],
                                spkr,
                                line,
                            ]
                            # print(spkr + ':' + line)
                            spkr = c[0]
                            line = c[1]
                    else:
                        line += c
            lines.loc[len(lines)] = [
                meta["Date"],
                meta["Start_Hour"],
                meta["Program"],
                meta["Title"],
                spkr,
                line,
            ]

            #########################
            ##Keyword and Sentiment##
            #########################

            lines['Line'] = lines['Line'].apply(lambda x: x.lower())
            lines["VADER"] = lines["Line"].apply(
                lambda x: list(sid.polarity_scores(x).values())[3]
            )

            word_sentiment_list = []
            w_list = []
            raw_text= ' '.join(lines['Line'])
            tokens = nltk.word_tokenize(raw_text)
            words = Counter(tokens).most_common()
            words1 = []
            i = 0
            while len(words1) < 15 and i < len(words):
                w = words[i][0]
                #print(w)
                tag =  nltk.pos_tag([w])
                t = tag[0][1] 
                if t in ['NN','NNS','NNP','NNPS','FW','SYM'] and w not in ['video','clip','i','end','begin']: #or t == "FW":
                    words1.append(w)
                i += 1

            for w in words1:
                try:
                    w1 = w
                    wdf = lines[lines['Line'].str.contains(w1)]
                    w_list.append(w1)
                    sent_list = list(wdf["VADER"])
                    word_sentiment_list.append(sent_list)
                except:
                    pass

                
            ######################
            ##Storing Each Topic##
            ######################
            for w in range(len(w_list)):
                if len(word_sentiment_list[w]) > 2:
                    tr_tp_st[0].append(l)
                    tr_tp_st[1].append(w_list[w])
                    tr_tp_st[2].append(np.mean(word_sentiment_list[w]))
                    tr_tp_st[3].append(lines["Program"][0])


    gl = pd.read_csv("CNNGuestList.csv", encoding="windows-1252")
    fox = open("cnn_text.txt", "r", encoding="windows-1252")
    fox = fox.readlines()
    for l in range(len(gl)):
        #######################################
        ##Turning transcript into df of lines##
        #######################################
        lines = pd.DataFrame(
            columns=["Date", "Start_Hour", "Program", "Title", "Speaker", "Line"]
        )
        # print(lines)
        transcript = fox[l]
        meta = gl.iloc[l, :]
        dt = meta["Date"]
        if month in str(dt):
            if randint(1, 100) <= 5:
                print(l)
            chunks = transcript.split("|")
            line = ""
            spkr = ""
            for c in chunks:
                if (
                    "Page" not in c[0:10]
                    and "...." not in c
                    and c[0:15] not in meta["Title"]
                ):
                    # print(c)
                    if ":" in c:
                        c = c.split(":")
                        if str.isupper(c[0]):
                            lines.loc[len(lines)] = [
                                meta["Date"],
                                meta["Start_Hour"],
                                meta["Program"],
                                meta["Title"],
                                spkr,
                                line,
                            ]
                            # print(spkr + ':' + line)
                            spkr = c[0]
                            line = c[1]
                    else:
                        line += c
            lines.loc[len(lines)] = [
                meta["Date"],
                meta["Start_Hour"],
                meta["Program"],
                meta["Title"],
                spkr,
                line,
            ]

            #########################
            ##Keyword and Sentiment##
            #########################

            lines['Line'] = lines['Line'].apply(lambda x: x.lower())
            lines["VADER"] = lines["Line"].apply(
                lambda x: list(sid.polarity_scores(x).values())[3]
            )

            word_sentiment_list = []
            w_list = []
            raw_text= ' '.join(lines['Line'])
            tokens = nltk.word_tokenize(raw_text)
            words = Counter(tokens).most_common()
            words1 = []
            i = 0
            while len(words1) < 15 and i < len(words):
                w = words[i][0]
                #print(w)
                tag =  nltk.pos_tag([w])
                t = tag[0][1] 
                if t in ['NN','NNS','NNP','NNPS','FW','SYM'] and w not in ['video','clip','i','end','begin']: #or t == "FW":
                    words1.append(w)
                i += 1

            for w in words1:
                try:
                    w1 = w
                    wdf = lines[lines['Line'].str.contains(w1)]
                    w_list.append(w1)
                    sent_list = list(wdf["VADER"])
                    word_sentiment_list.append(sent_list)
                except:
                    pass

                
            ######################
            ##Storing Each Topic##
            ######################
            for w in range(len(w_list)):
                if len(word_sentiment_list[w]) > 2:
                    tr_tp_st[0].append(l)
                    tr_tp_st[1].append(w_list[w])
                    tr_tp_st[2].append(np.mean(word_sentiment_list[w]))
                    tr_tp_st[3].append(lines["Program"][0])
    return tr_tp_st

In [4]:
# Save out processed and topic modeled data
months = ['January','February','March','April','May','June','July','August','September','October','November','December']
for m in months:
    tr_tp_st = transcript_cleaning(m)
    tdf = pd.DataFrame(zip(tr_tp_st[0], tr_tp_st[1], tr_tp_st[2], tr_tp_st[3]), 
                       columns=["Transcript", "Topic", "Sentiment", "Program"],)
    with open('v9 processed_transcripts_' + m + '2020.pkl', 'wb') as file:
        pickle.dump(tr_tp_st, file)
    tdf.to_csv('v9 processed_transcripts_' + m + '2020.csv')

85
496
859
891
1286
1295
1612
1798
1859
2293
2451
2675
2755
827
932
1407
1628
1631
2130
233
403
881
974
1746
1783
2121
2132
2225
2236
2305
2363
2642
3011
3554
3682
3684
3746
3854
3861
3871
3890
4252
4615
4622
4865
5205
5885
5887
5891
5933
6037
6039
6420
6697
7011
7031
7243
7300
7412
7751
7753
7850
8129
8248
9191
9943
10179
10184
10195
10786
10819
544
908
938
1499
1633
2620
2695
84
85
447
531
783
820
1008
1611
1847
1851
728
896
928
1107
1214
1325
1501
1516
1691
1917
2159
2197
2227
2435
2530
2663
3225
3559
4119
4133
4493
4508
4830
5338
5862
5906
6146
6162
6755
6782
6945
6993
7004
7111
7764
8166
8524
9942
10374
182
203
226
399
1047
2353
2522
2553
147
433
875
906
975
1213
1466
1475
1922
1931
186
293
336
611
865
1459
1708
1722
1826
1843
1995
3047
3641
3863
3962
4342
5560
5626
5846
5955
6136
6137
6151
6857
6932
6943
7012
7269
7289
7765
8297
8617
9269
9560
9679
10354
10422
10780
390
1005
1281
1723
2637
2746
642
644
790
801
1089
1580
1585
1787
1804
1159
1228
1232
1815
2008
2080
2471
2581
2614


In [7]:
print(tr_tp_st[0][:5])
print(tr_tp_st[1][:5])
print(tr_tp_st[2][:5])
print(tr_tp_st[3][:5])
print(len(tr_tp_st[1]))
print(tr_tp_st[3][1440])

[21, 21, 21, 21, 21]
[['generation', 'free', 'vote', 'people', 'man', 'feel', 'see'], ['know', 'word', 'impeachment', 'election', 'evidence', 'use'], ['stage', 'mueller', 'grief', 'question', 'country'], ['information', 'president', 'memo', 'medium', 'leak', 'news', 'receive'], ['tonight', 'liberal', 'way', 'angle', 'good']]
[0.1352142857142857, 0.004306896551724135, -0.026888000000000006, -0.18042962962962963, -0.03984999999999999]
['Fox News Network INGRAHAM ANGLE', 'Fox News Network INGRAHAM ANGLE', 'Fox News Network INGRAHAM ANGLE', 'Fox News Network INGRAHAM ANGLE', 'Fox News Network INGRAHAM ANGLE']
6571
MSNBC 11TH HOUR WITH BRIAN WILLIAMS
